In [21]:
from matplotlib import pyplot
import numpy as np
import os
import torch
import torchvision.transforms.functional as fn
import torchvision
from torchvision.io import read_image
from PIL import Image
from torchvision.transforms import ToTensor
from torch.nn.functional import one_hot as one_hot_encoding
import matplotlib.pyplot as pltd
import import_ipynb
from CNN_model import Net
from torchsummary import summary

In [22]:
path = "/Users/summer/desktop/machine learning project/data/UCMerced_LandUse/Images"
category = os.listdir(path)

In [23]:
category_names = [
    'agricultural',
    'forest',
    'beach',
    'runway',
    'river',
]
num_images = 100
nb_classes = len(category_names)
patch_size = 256
channels = 3

In [24]:
# # some experiments 
# new_image = Image.open("/Users/summer/desktop/machine learning project/data/UCMerced_LandUse/Images/agricultural/agricultural00.tif")
# image = ToTensor()(new_image)
# print(image.shape)
# new_image = fn.resize(image, size=[patch_size//4, patch_size//4])

In [25]:
# first we take out the data and transform them 
X = []
y = []

for i in range(len(category_names)):
    for j in range(num_images):
        image = Image.open("/Users/summer/desktop/machine learning project/data/UCMerced_LandUse/Images/{}/{}{:02d}.tif".format(category_names[i],category_names[i],j))
        
        # make it a tensor
        image = ToTensor()(image)
        # resize it 
        image = fn.resize(image, size=[patch_size//4, patch_size//4])

        image_arr = np.asarray(image)

        X.append(image_arr)
        y.append([i])

        

X = np.array(X)
y = np.array(y)
X = torch.from_numpy(X)
y = torch.from_numpy(y)

# we don't want labels to be 1, 2 ,3 ,
# we want it in the format of one hot encoding
y = one_hot_encoding(y)

torch.save(X, "/Users/summer/desktop/machine learning project/datasets.pt")
torch.save(y, "/Users/summer/desktop/machine learning project/labels.pt")


In [26]:
# now we have to split the dataset into training, validation and testing 
dataset_length = X.shape[0]
shuffled_indices = np.arange(dataset_length)

In [27]:
np.random.seed(100)
np.random.shuffle(shuffled_indices)

In [28]:
training_indices, validation_indices, test_indices = shuffled_indices[0:int(dataset_length*0.6)],\
                                                     shuffled_indices[int(dataset_length*0.6):int(dataset_length * 0.8)],\
                                                     shuffled_indices[int(dataset_length * 0.8):]

In [33]:
training_set, validation_set, test_set = X[training_indices],X[validation_indices],X[test_indices]
training_label, validation_label, test_label = y[training_indices], y[validation_indices], y[test_indices]
print(training_set.shape)

torch.Size([300, 3, 64, 64])


In [47]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, (3, 3),padding="same")
        self.conv2 = nn.Conv2d(32, 64, (3,3),padding='same')
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d((2,2), stride=(2,2))
        self.fc = nn.Linear(16384, 5)
        self.dropout = nn.Dropout(p=0.5)
        self.flatten = nn.Flatten()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.flatten(x) # flatten all dimensions except batch
        x = self.fc(x)
        x = self.dropout(x)
        x = self.softmax(x)
        
        return x
    

In [114]:
model = Net()
output = model.forward(training_set)
int_label = torch.argmax(training_label, dim=2).reshape(300,)
print(int_label)

tensor([0, 0, 4, 3, 0, 1, 4, 1, 1, 1, 1, 0, 1, 4, 1, 4, 3, 2, 1, 4, 1, 4, 0, 2,
        2, 2, 1, 3, 2, 0, 1, 3, 4, 3, 0, 3, 4, 1, 0, 0, 1, 4, 2, 2, 1, 2, 4, 4,
        4, 3, 4, 3, 3, 4, 0, 2, 4, 0, 4, 4, 2, 3, 1, 1, 3, 1, 1, 3, 2, 1, 0, 3,
        3, 0, 0, 3, 2, 2, 0, 4, 0, 1, 4, 3, 2, 0, 0, 0, 1, 4, 1, 3, 3, 4, 1, 4,
        1, 1, 0, 3, 2, 4, 3, 4, 0, 1, 1, 0, 4, 0, 3, 4, 4, 4, 4, 3, 2, 2, 3, 2,
        0, 0, 0, 3, 4, 2, 3, 3, 0, 2, 0, 1, 2, 4, 0, 0, 2, 1, 3, 4, 0, 3, 2, 1,
        4, 3, 1, 4, 0, 3, 1, 0, 3, 4, 1, 4, 2, 4, 2, 0, 0, 2, 2, 4, 1, 2, 4, 3,
        4, 0, 1, 2, 2, 0, 4, 1, 4, 0, 1, 1, 3, 1, 2, 4, 4, 1, 1, 0, 3, 4, 4, 4,
        4, 3, 3, 1, 2, 0, 2, 0, 4, 2, 3, 1, 2, 0, 1, 0, 3, 3, 4, 0, 1, 0, 3, 3,
        1, 4, 2, 4, 3, 4, 4, 2, 2, 1, 2, 0, 0, 0, 4, 0, 2, 3, 3, 4, 2, 0, 3, 2,
        2, 3, 4, 0, 4, 3, 4, 2, 1, 2, 0, 3, 2, 1, 1, 1, 3, 3, 3, 3, 2, 2, 0, 2,
        1, 4, 0, 3, 4, 1, 3, 0, 2, 1, 0, 1, 4, 1, 1, 3, 4, 3, 2, 1, 2, 1, 2, 2,
        0, 0, 3, 4, 4, 4, 4, 3, 4, 2, 1,

In [116]:
loss = nn.CrossEntropyLoss()
print(loss(output, int_label))

tensor(1.6089, grad_fn=<NllLossBackward0>)


In [106]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
print(input)
target = torch.empty(3, dtype=torch.long).random_(5)
print(target)
output = loss(input, target)
output.backward()


tensor([[-0.4083, -0.2705,  1.4404,  0.2484,  0.5575],
        [-0.3670, -0.9370,  2.2625, -0.3673, -2.1630],
        [ 0.2162,  0.8056,  0.4344, -1.2499,  1.0685]], requires_grad=True)
tensor([2, 3, 0])
